# Senegal data transformation
- move the layer to the side
- clipp the layer
- reclassify the content of the file
- reproject the data

UNFINISHED

In [1]:
#Import the libraries
import os
import sys
from osgeo import gdal
from osgeo import ogr
import numpy as np
import glob
import pandas as pd
import geopandas as gpd


gdal.UseExceptions()

In [2]:
"""We specify the current working direcroty"""
# os.chdir(r"//akif.internal/public/z_resources/hydrosoil/hihydrosoil_ruben") #works fine


#Get a list of the two files inside the folder
path = r"//akif.internal/public/demo-geo-stack/geoserver/data/nca_senegal_land_cover_10m_26_classes/workspace"
File_list = [] #f for f in os.listdir(path) if os.isfile(mypath,f)
for file in os.listdir(path):
    if file.endswith(".tif"):
        if file not in File_list:
            File_list.append(file)
    else:
        pass
print(File_list)


['land_cover_10m_2010.tif', 'land_cover_10m_2015.tif', 'land_cover_10m_2015_moved.tif']


In [ ]:
"""We are going to move the layer 2015 2'4m to the side, so both 2010 and 2015 layers are equal, they are both 10m pixel size"""
#Source: https://gis.stackexchange.com/questions/13561/how-to-translate-reposition-a-raster-in-python
os.chdir(r"//akif.internal/public/demo-geo-stack/geoserver/data/nca_senegal_land_cover_10m_26_classes/workspace")

# Open in read/write mode
ds_2015 = gdal.Open("land_cover_10m_2015.tif", 1) # 0=Read-only, 1=Read-Write

# Get affine transform coefficients
gt = ds_2015.GetGeoTransform()
print(gt) #example (2776450.0, 100.0, 0.0, 6352650.0, 0.0, -100.0)
#The geotransform gt object is a 6-parameter tuple described here. You want to edit the first and fourth items:

# Convert tuple to list, so we can modify it
gtl = list(gt)
#change the values
gtl[0] -= 24.0  # Move west 2,4 km / 1000 = 1km
# Example [2777450.0, 100.0, 0.0, 6332650.0, 0.0, -100.0]


#Source: https://gis.stackexchange.com/questions/75891/in-python-reading-a-gdal-raster-from-memory-instead-of-a-file/75893
# we real the original file, and do the corresponding transformation
dst_filename = "land_cover_10m_2015_moved.tif"
driver = gdal.GetDriverByName('GTif') #We set here the format, MEM/GTif
dst_ds = driver.CreateCopy( dst_filename, ds_2015, 0,
            # outputType = gdal.GDT_Int16, 
            options = ['COMPRESS=DEFLATE','TILED=YES','BLOCKXSIZE=128','BLOCKYSIZE=128','BIGTIFF=YES','COPY_SRC_OVERVIEWS=YES'] )
# Save the geotransform to the raster
dst_ds.SetGeoTransform(tuple(gtl))


dst_ds = None # equivalent to save/close

In [ ]:
"""Here we do the clipping part"""
os.chdir(r"//akif.internal/public/demo-geo-stack/geoserver/data/nca_senegal_land_cover_10m_26_classes/workspace")

os.chdir(r"//akif.internal/public/demo-geo-stack/geoserver/data/nca_senegal_land_cover_10m_26_classes/workspace")
#ds_2015 is already opened
ds_2010 = gdal.Open("land_cover_10m_2010.tif", 1) # 0=Read-only, 1=Read-Write

# This is to elaborate the table we parse with the SQL sentence
vector_ds = ogr.GetDriverByName('ESRI Shapefile').CreateDataSource("senegal_au_fixed.shp")
lyr = vector_ds.CreateLayer('cutline')
f = ogr.Feature(lyr.GetLayerDefn())
lyr.CreateFeature(f)
f = None
# vector_ds = None

sql = "SELECT * FROM cutline" #Build the OGR SQL

# Clip the input Raster

ds_2010_clipped = gdal.Warp(ds_2010, 
        format = 'MEM', 
        cutlineDSName="senegal_au_fixed.shp",
        cutlineSQL=sql,
        # cropToCutline=True,
        srcDSOrSrcDSTab=ds_2010,
        creationOptions = ['COMPRESS=DEFLATE','TILED=YES','BLOCKXSIZE=128','BLOCKYSIZE=128','BIGTIFF=YES','COPY_SRC_OVERVIEWS=YES']) 
        #RuntimeError: Failed to identify source layer from datasource. 

ds_2015_clipped = gdal.Warp(ds_2015, 
        format = 'MEM', 
        cutlineDSName="senegal_au_fixed.shp",
        cutlineSQL=sql,
        # cropToCutline=True,
        creationOptions = ['COMPRESS=DEFLATE','TILED=YES','BLOCKXSIZE=128','BLOCKYSIZE=128','BIGTIFF=YES','COPY_SRC_OVERVIEWS=YES'])

ogr.GetDriverByName('ESRI Shapefile').DeleteDataSource("senegal_au_fixed.shp")

ds_2015 = None 
ds_2010 = None



Error: Session cannot generate requests

In [5]:
os.chdir(r"//akif.internal/public/demo-geo-stack/geoserver/data/nca_senegal_land_cover_10m_26_classes/workspace")



# This is to elaborate the table we parse with the SQL sentence
vector_ds = ogr.GetDriverByName('ESRI Shapefile').CreateDataSource("senegal_au_fixed.shp")
lyr = vector_ds.CreateLayer('cutline')

In [ ]:
"""Here we do the reclassification"""

